## Simple Pipeline

In [1]:
import openai
import yaml
from src import utils
import time
from pprint import pprint
import os
import json
import argparse
from tqdm import tqdm
import re
import requests

In [2]:
## Test dataset

# GSM8K 8792 (7473/1319)
gsm8k = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

# MWP_KR_DATA 3750
mwp_kr = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning-test.jsonl'

# AddSub 395
addsub = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_addsub-test.json'

# MultiArith 600
multiarith = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_multiarith-test.json'

# SingleEq 395
single_eq = '/userhomes/philhoon/kt-ai-challenge/eng_data/kor-D_single_eq-test.json'

In [3]:
gsm8k_data = utils.read_jsonlines(gsm8k)
mwp_kr_data = utils.read_jsonlines(mwp_kr)
addsub_data = utils.read_jsonlines(addsub)
multiarith_data = utils.read_jsonlines(multiarith)
single_eq_data = utils.read_jsonlines(single_eq)

In [4]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [5]:
api_kt = '/userhomes/philhoon/kt-ai-challenge/api-peft3.json'
api_kt = utils.read_json_file(api_kt)

In [6]:
model_kt = model_api(api_kt['url'], api_kt['headers'])
BODY_TEMP = api_kt['body']

In [7]:
api = 'openai'
api_key_file = '/userhomes/philhoon/kt-ai-challenge/api-openai.yaml'

In [8]:
# model_name="gpt-3.5-turbo-0301",
model_name = "gpt-3.5-turbo-0613"

max_token=2048
temperature=0.0

In [9]:
input_file = single_eq
data = utils.read_jsonlines(input_file)

In [10]:
# openai key
key_dict = utils.get_keys(api_key_file)
openai.api_key = key_dict[api]

In [11]:
# output_file = '/userhomes/philhoon/kt-ai-challenge/data/public_mwp_data_v2_reasoning.jsonl'
# print(f'output_file : {output_file}')

In [12]:
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-inf-1.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl

In [13]:
# output_file = output_path + '/' + 'gsm8k-korean.jsonl'
# print(output_file)

In [14]:
# Jsonlines writier
# jsonl_writer = utils.JSONLWriter(output_file)

In [15]:
# data = utils.read_json_file(input_file)

In [16]:
from pprint import pprint

In [17]:
# data = data['data']

In [18]:
def remove_line_split(context):
    context = re.sub(r'\n{1,}', ' ', context)
    context = re.sub(r'\s{2,}', ' ', context)
    return context

In [19]:
def generate_answer(message, model_name, max_token, temperature):
    """
    Generate response
    """
    try:
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=message,
            temperature=temperature,
            max_tokens=max_token,
            n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(message, model_name, max_token, temperature)

    return completion

In [20]:
def get_kor_msg(context):
    """
    Message for chat api
    """

    msg = [
        {
            "role" : "system",
            "content" : "다음 질문에 차근차근 답하시오."
        },
        {
            "role" : "user",
            "content" : context
        }
    ]
    
    return msg

In [21]:
from tqdm import tqdm

In [22]:
def get_kt_verfiy(que, ans):
    prompt = f'해당 질문에 대한 답변이 맞는지 확인하세요 질문:{que} 답변:{ans}'
    return prompt

In [23]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [24]:
cnt = 0
for ins in tqdm(data):
    if 'kor_question' in ins:
        kor_que = ins['kor_question']
    else:
        kor_que = ins['question']
        
    if 'answer_s' in ins:
        answer = ins['answer_s']
    else:
        answer = ins['answer']
    
    query = remove_line_split(kor_que)
    
    query_msg = get_kor_msg(query)

    query_completion = generate_answer(query_msg, model_name, max_token, temperature)
    q_response = query_completion["choices"][0]["message"]["content"]
    
    
    q_response = remove_line_split(q_response)
    
    print(f"query_msg: \n{query_msg}")
    print(f"q_response: \n{q_response}")
    
    print('----------------------------')
    prompt = get_kt_verfiy(kor_que, q_response)
    print(f"prompt : \n{prompt}")
    body = get_body(prompt, BODY_TEMP)
    response = model_kt.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    print(f"response : \n{res_context}")
    print(f"----------------------------")
    print(f"answer : \n{answer}")
    print(f"============================")
    

#     ans_completion = generate_answer(ans_msg, model_name, max_token, temperature)
#     ans_response = ans_completion["choices"][0]["message"]["content"]
    
#     instance['kor_query'] = q_response
#     instance['kor_ans'] = ans_response
#     pprint(instance)
#     jsonl_writer.write_json_line(ins)a
    if cnt == 5:
        break
    else:
        cnt+=1

  0%|                                                                                                                                                | 0/102 [00:00<?, ?it/s]

query_msg: 
[{'role': 'system', 'content': '다음 질문에 차근차근 답하시오.'}, {'role': 'user', 'content': '헛간에는 28개의 건초 더미가 있었습니다. 팀은 오늘 헛간에 더 많은 건초 더미를 쌓았습니다. 이제 헛간에는 54개의 건초 더미가 있습니다. 그는 헛간에 몇 개의 건초 더미를 저장했을까요?'}]
q_response: 
팀이 헛간에 쌓은 건초 더미의 개수를 알기 위해서는 헛간에 있던 건초 더미의 개수를 빼야 합니다. 헛간에 있던 건초 더미의 개수는 54개 - 28개 = 26개입니다. 따라서, 팀은 헛간에 26개의 건초 더미를 저장했습니다.
----------------------------
prompt : 
해당 질문에 대한 답변이 맞는지 확인하세요 질문:헛간에는 28개의 건초 더미가 있었습니다. 팀은 오늘 헛간에 더 많은 건초 더미를 쌓았습니다. 이제 헛간에는 54개의 건초 더미가 있습니다. 그는 헛간에 몇 개의 건초 더미를 저장했을까요? 답변:팀이 헛간에 쌓은 건초 더미의 개수를 알기 위해서는 헛간에 있던 건초 더미의 개수를 빼야 합니다. 헛간에 있던 건초 더미의 개수는 54개 - 28개 = 26개입니다. 따라서, 팀은 헛간에 26개의 건초 더미를 저장했습니다.


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  1%|█▎                                                                                                                                      | 1/102 [00:09<16:15,  9.66s/it]

response : 
"그는 헛간에 28개의 건초 더미를 쌓았습니다. 팀은 오늘 헛간에 더 많은 건초 더미를 쌓았습니다. 이제 헛간에는 54개의 건초 더미가 있습니다. 그는 헛간에 몇 개의 건초 더미를 저장했을까요?"
----------------------------
answer : 
26.0
query_msg: 
[{'role': 'system', 'content': '다음 질문에 차근차근 답하시오.'}, {'role': 'user', 'content': '레스토랑에서 식사를 한 후, 샐리, 샘, 앨리사는 계산서를 균등하게 나누기로 결정했습니다. 각각의 사람이 45달러를 지불했다면, 총 계산서는 얼마인가요?'}]
q_response: 
세 사람이 총 45달러를 지불했으므로, 세 사람의 지불 금액을 합산하면 총 계산서 금액을 구할 수 있습니다. 45달러 x 3 = 135달러 따라서, 총 계산서 금액은 135달러입니다.
----------------------------
prompt : 
해당 질문에 대한 답변이 맞는지 확인하세요 질문:레스토랑에서 식사를 한 후, 샐리, 샘, 앨리사는 계산서를 균등하게 나누기로 결정했습니다. 각각의 사람이 45달러를 지불했다면, 총 계산서는 얼마인가요? 답변:세 사람이 총 45달러를 지불했으므로, 세 사람의 지불 금액을 합산하면 총 계산서 금액을 구할 수 있습니다. 45달러 x 3 = 135달러 따라서, 총 계산서 금액은 135달러입니다.


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  2%|██▋                                                                                                                                     | 2/102 [00:17<13:54,  8.35s/it]

...

response : 
"사라의 고등학교는 4경기를 패배했으므로
----------------------------
answer : 
8.0
query_msg: 
[{'role': 'system', 'content': '다음 질문에 차근차근 답하시오.'}, {'role': 'user', 'content': '현재 공원에는 호두나무가 22그루 있습니다. 공원 직원들은 오늘 더 많은 호두나무를 심을 예정입니다. 작업이 끝나면 공원에는 총 55그루의 호두나무가 있을 것입니다. 공원 직원들은 오늘 몇 그루의 호두나무를 심었을까요?'}]
q_response: 
공원에 심을 호두나무의 수를 x라고 가정하겠습니다. 현재 공원에 있는 호두나무의 수는 22그루이므로, 작업이 끝난 후에는 22 + x그루의 호두나무가 있을 것입니다. 작업이 끝난 후에는 총 55그루의 호두나무가 있을 것이므로, 다음 식을 세울 수 있습니다. 22 + x = 55 이 식을 풀면, x = 55 - 22 x = 33 따라서, 공원 직원들은 오늘 33그루의 호두나무를 심었습니다.
----------------------------
prompt : 
해당 질문에 대한 답변이 맞는지 확인하세요 질문:현재 공원에는 호두나무가 22그루 있습니다. 공원 직원들은 오늘 더 많은 호두나무를 심을 예정입니다. 작업이 끝나면 공원에는 총 55그루의 호두나무가 있을 것입니다. 공원 직원들은 오늘 몇 그루의 호두나무를 심었을까요? 답변:공원에 심을 호두나무의 수를 x라고 가정하겠습니다. 현재 공원에 있는 호두나무의 수는 22그루이므로, 작업이 끝난 후에는 22 + x그루의 호두나무가 있을 것입니다. 작업이 끝난 후에는 총 55그루의 호두나무가 있을 것이므로, 다음 식을 세울 수 있습니다. 22 + x = 55 이 식을 풀면, x = 55 - 22 x = 33 따라서, 공원 직원들은 오늘 33그루의 호두나무를 심었습니다.


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  5%|██████▋                                                                                                                                 | 5/102 [00:38<12:22,  7.65s/it]

response : 
"공원 직원들이 오늘 더 많은 호두나무를 심을 예정이므로
----------------------------
answer : 
33.0
query_msg: 
[{'role': 'system', 'content': '다음 질문에 차근차근 답하시오.'}, {'role': 'user', 'content': '앨리사는 용돈의 절반을 영화 보는 데 사용했습니다. 그녀는 가족 차량을 세척하여 8달러를 벌었습니다. 그렇다면 그녀의 주간 용돈은 얼마인가요?'}]
q_response: 
앨리사가 영화를 보는 데 사용한 돈의 절반은 8달러입니다. 따라서 영화를 보는 데 사용한 돈은 8달러 * 2 = 16달러입니다. 앨리사의 주간 용돈은 영화를 보는 데 사용한 돈의 2배인 16달러입니다.
----------------------------
prompt : 
해당 질문에 대한 답변이 맞는지 확인하세요 질문:앨리사는 용돈의 절반을 영화 보는 데 사용했습니다. 그녀는 가족 차량을 세척하여 8달러를 벌었습니다. 그렇다면 그녀의 주간 용돈은 얼마인가요? 답변:앨리사가 영화를 보는 데 사용한 돈의 절반은 8달러입니다. 따라서 영화를 보는 데 사용한 돈은 8달러 * 2 = 16달러입니다. 앨리사의 주간 용돈은 영화를 보는 데 사용한 돈의 2배인 16달러입니다.


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  5%|██████▋                                                                                                                                 | 5/102 [00:44<14:16,  8.83s/it]

response : 
"앨리사의 주간 용돈은 8*2=16달러입니다. 그러므로
----------------------------
answer : 
8.0


In [ ]:
# body = get_body('hi', BODY_TEMP)

In [ ]:
# response = model_kt.generate(body)

In [ ]:
response = requests.post(api_kt['url'], data=body, headers=api_kt['headers'], verify=False)/